In [43]:
import pandas as pd

In [44]:
df = pd.read_csv("AirPassengers.csv")

In [45]:
#show me first five records 
df.head()

,Month,#Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [46]:
# show me last five records
df.tail()

,Month,#Passengers
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390
143,1960-12,432


In [47]:
#dimention of the dataset
df.shape

(144, 2)

In [48]:
df.columns

Index(['Month', '#Passengers'], dtype='object')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Month        144 non-null    object
 1   #Passengers  144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [50]:
df.describe()

,#Passengers
count,144.000000
mean,280.298611
std,119.966317
min,104.000000
25%,180.000000
50%,265.500000
75%,360.500000
max,622.000000


In [51]:
df.isnull().sum()

Month          0
#Passengers    0
dtype: int64

In [52]:
#converting date column data type to date time
df["Date"] = pd.to_datetime(df["Month"])
df=df.drop(["Month"],axis=1)
df.head()

,#Passengers,Date
0,112,1949-01-01
1,118,1949-02-01
2,132,1949-03-01
3,129,1949-04-01
4,121,1949-05-01


In [53]:
df_new = df.set_index("Date")

In [56]:
df_new.head()

,#Passengers
Date,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [65]:
#spliting dataset to train and test sets
split_date = '01-Dec-1957'
df_train = df_new.loc[df_new.index <= split_date].copy()
df_test = df_new.loc[df_new.index > split_date].copy()

In [68]:
print(f"Traing set shape  : {df_train.shape}\nTesting set shape : {df_test.shape}")

Traing set shape  : (108, 1)
Testing set shape : (36, 1)
